# LuxendoManager Example Notebook

In [1]:
# --- Imports ---
import pymif.microscope_manager as mm
import napari
import os
import time

In [2]:
# --- Set input data path ---
main_path = "/g/mif/people/gritti/code/pymif_test_data"
dataset_name = "luxendo"

main_path = "/g/mif/common/Data/Z_from_LCS_24-07-16/Coll_Montse/MouseEmbryo_micemetyleneBlue/2022-08-09_132304/processed"
dataset_name = "20221108-162726_Task_2_LM_FullRes_C"

# main_path = "/mif-users/Users/Arias_Adrian/scans_CSL_P21/2400_31573_s"
# dataset_name = "20250523-105843_Task_1_stitching_1_C"

luxendo_path = f"{main_path}/{dataset_name}"

In [3]:
# --- Initialize manager ---
# Here, the argument chunks can be used to specify a preferred chunk (Default: (1,1,16,256,256)
dataset = mm.LuxendoManager(path=luxendo_path, chunks=(1,1,32,2048,2048))

In [4]:
# --- Show metadata summary ---
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")
print("CHUNK SIZE:", dataset.chunks)

SIZE: [(1, 2, 920, 6646, 4597), (1, 2, 480, 3328, 2304), (1, 2, 256, 1664, 1152), (1, 2, 128, 832, 576), (1, 2, 64, 416, 288)]
SCALES: [(5.0, 2.925, 2.925), (10.0, 5.85, 5.85), (20.0, 11.7, 11.7), (40.0, 23.4, 23.4), (80.0, 46.8, 46.8)]
UNITS: ('micrometer', 'micrometer', 'micrometer')
TIME_INCREMENT: 1.0
TIME_INCREMENT_UNIT: s
CHANNEL_NAMES: ['tp:0_ch:6_st:3-x00-y00-3-x00-y01_ang:h0-v90_obj:bottom-bottom_cam:bottom', 'tp:0_ch:7_st:3-x00-y00-3-x00-y01_ang:h0-v90_obj:bottom-bottom_cam:bottom']
CHANNEL_COLORS: [16711680, 255]
DTYPE: uint16
PLANE_FILES: None
AXES: tczyx
CHUNK SIZE: (1, 1, 32, 2048, 2048)


In [5]:
dataset.data[0]

dask.array<stack, shape=(1, 2, 920, 6646, 4597), dtype=uint16, chunksize=(1, 1, 32, 2048, 2048), chunktype=numpy.ndarray>

In [ ]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
# It reads only the visualizing slice. Faster to open but not interactive.
# NOT RECOMMENDED FOR LARGE h5 DATASETS!
# To visualize lower resolution, use the next cell.
# To visualize the whole dataset at full resolution, convert data into ome-zarr

# start = time.time()
# viewer = dataset.visualize()
# print(f"{time.time()-start}")

In [ ]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
# The start_level-stop_level options, generate lower resolution layers and load them into memory
# NOTE: This works because the dataset has 5 resolution layers
# It takes some time because it has to open the files and read the corresponding layers
# Best would be to resave the dataset in ome-zarr (see next cell on how to do it)

start = time.time()
viewer = dataset.visualize( 
                        start_level = 5, 
                        stop_level = 6, 
                        in_memory = True 
                    )
print(f"{time.time()-start}")

In [ ]:
# --- Build pyramid if not already ---
dataset.build_pyramid(
                    num_levels=8, 
                    downscale_factor=2,
                    start_level=0
                    )

# # --- Optional: Write to OME-Zarr format ---
# output_zarr_path = f"{main_path}/{dataset_name}.zarr"
# dataset.write(output_zarr_path)

In [ ]:
# --- Show metadata summary for updated dataset ---
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")
print("CHUNK SIZE:", dataset.chunks)

# Now we use the ZarrManager to load the new data and display it

In [6]:
input_zarr_path = f"{main_path}/{dataset_name}.zarr"

In [7]:
# --- Initialize manager ---
# Here, the argument chunks can be used to specify a preferred chunk (Default: (1,1,16,256,256)
# Here, we use the same chunks.
zarr_dataset = mm.ZarrManager(path=input_zarr_path, chunks=(1,1,32,2048,2048))

In [8]:
for i in dataset.metadata:
    print(f"{i.upper()}: {zarr_dataset.metadata[i]}")
print("CHUNK SIZE:", zarr_dataset.chunks)

SIZE: [(1, 2, 920, 6646, 4597), (1, 2, 460, 3323, 2299), (1, 2, 230, 1662, 1150), (1, 2, 115, 831, 575), (1, 2, 58, 416, 288), (1, 2, 29, 208, 144)]
SCALES: [(5.0, 2.925, 2.925), (10.0, 5.85, 5.85), (20.0, 11.7, 11.7), (40.0, 23.4, 23.4), (80.0, 46.8, 46.8), (160.0, 93.6, 93.6)]
UNITS: ('micrometer', 'micrometer', 'micrometer')
TIME_INCREMENT: 1.0
TIME_INCREMENT_UNIT: s
CHANNEL_NAMES: ['tp:0_ch:6_st:3-x00-y00-3-x00-y01_ang:h0-v90_obj:bottom-bottom_cam:bottom', 'tp:0_ch:7_st:3-x00-y00-3-x00-y01_ang:h0-v90_obj:bottom-bottom_cam:bottom']
CHANNEL_COLORS: [16711680, 255]
DTYPE: uint16
PLANE_FILES: None
AXES: tczyx
CHUNK SIZE: (1, 1, 32, 2048, 2048)


In [11]:
zarr_dataset.data[1]

dask.array<rechunk-merge, shape=(1, 2, 460, 3323, 2299), dtype=uint16, chunksize=(1, 1, 32, 2048, 2048), chunktype=numpy.ndarray>

In [13]:
# --- Visualize in Napari ---
# Opens napari with pyramid-aware viewing and color channels
# It reads only the visualizing slice. Faster but not interactive.
start = time.time()
viewer = zarr_dataset.visualize()
print(f"{time.time()-start}")

1.3362369537353516


In [17]:
start = time.time()
# zarr_dataset.data = [level.rechunk((1, 1, 32, 512, 512)) for level in zarr_dataset.data]
viewer = zarr_dataset.visualize(                        
                        start_level = 0, 
                        stop_level = 2, 
                        in_memory = False 
)
print(f"{time.time()-start}")

27.364320039749146


Traceback (most recent call last):
  File "/home/gritti/.local/lib/python3.10/site-packages/napari/utils/action_manager.py", line 230, in _trigger
    self.trigger(name)
  File "/home/gritti/.local/lib/python3.10/site-packages/napari/utils/action_manager.py", line 432, in trigger
    return self._actions[name].injected()
  File "/home/gritti/.local/lib/python3.10/site-packages/in_n_out/_store.py", line 804, in _exec
    result = func(**bound.arguments)
  File "/home/gritti/.local/lib/python3.10/site-packages/napari/components/_viewer_key_bindings.py", line 66, in toggle_ndisplay
    viewer.dims.ndisplay = 3
  File "/home/gritti/.local/lib/python3.10/site-packages/napari/utils/_proxies.py", line 147, in __setattr__
    setattr(self.__wrapped__, name, value)
  File "/home/gritti/.local/lib/python3.10/site-packages/napari/utils/events/evented_model.py", line 308, in __setattr__
    with ComparisonDelayer(self):
  File "/home/gritti/.local/lib/python3.10/site-packages/napari/utils/events/e

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
File ~/.local/lib/python3.10/site-packages/napari/_qt/threads/status_checker.py:126, in StatusChecker.calculate_status(self=<napari._qt.threads.status_checker.StatusChecker object>)
    122     return
    124 try:
    125     # Calculate the status change from cursor's movement
--> 126     res = viewer._calc_status_from_cursor()
        viewer = Viewer(camera=Camera(center=(0.0, 9718.3125, 6721.65), zoom=0.04919524028762668, angles=(0.0, 0.0, 89.99999999999999), perspective=0.0, mouse_pan=True, mouse_zoom=True, orientation=(<DepthAxisOrientation.TOWARDS: 'towards'>, <VerticalAxisOrientation.DOWN: 'down'>, <HorizontalAxisOrientation.RIGHT: 'right'>)), cursor=Cursor(position=(0.0, 2.0848393516093872e-13, 13123.113546409353, 3235.540417850378), scaled=True, style=<CursorStyle.STANDARD: 'standard'>, size=1.0), dims=Dims(ndim=4, ndisplay=3, 